In [ ]:
import numpy as np
import compas.geometry as cg
from compas.datastructures import Mesh
from compas_cgal.triangulation import refined_delaunay_mesh as rdm
from compas_cgal.meshing import trimesh_remesh
from compas_notebook.viewer import Viewer

## 1. Define Geometry (Boundaries)

We define an outer circular boundary and several inner circular holes. Each hole has specific parameters for position, radius, height, and tilt.

In [ ]:
# Outer boundary (The base of the volcanoes)
radius_outer = 20
boundary = cg.Polygon.from_sides_and_radius_xy(60, radius_outer)
boundary_coords = np.array(boundary, dtype=np.float64)

# Inner boundaries (The craters/holes)
# Define parameters for each hole: position, radius, height, tilt_angle (degrees)
hole_params = [
    {'pos': [8, 0, 0],  'radius': 5.0, 'height': 8.0,  'tilt_angle': 45},   # Up, tilt outward
    {'pos': [-8, 0, 0], 'radius': 2.5, 'height': 12.0,  'tilt_angle': 30},   # Up
    {'pos': [0, 8, 0],  'radius': 1.8, 'height': -15.0, 'tilt_angle': 20},   # Down
    {'pos': [0, -8, 0], 'radius': 3, 'height': -7.0, 'tilt_angle': 60},   # Down
]

holes = []
hole_polygons = [] # Keep compas polygons for viz
for params in hole_params:
    # Create hole polygon
    poly = cg.Polygon.from_sides_and_radius_xy(30, params['radius'])
    # Translate to position
    T = cg.Translation.from_vector(params['pos'])
    h = poly.transformed(T)
    hole_polygons.append(h)
    holes.append(np.array(h, dtype=np.float64))

# Visualize Boundaries
viewer = Viewer()
viewer.scene.clear()
viewer.scene.add(boundary, name="Outer Boundary")
for i, h in enumerate(hole_polygons):
    viewer.scene.add(h, name=f"Hole {i}")
viewer.show()

## 2. Generate Mesh

We use `refined_delaunay_mesh` from `compas_cgal` to generate a high-quality triangular mesh within the boundaries.

In [ ]:
# Use refined delaunay mesh to get a good quality mesh
# We pass the boundary and the holes.
# maxlength controls the density of the mesh.
V, F = rdm(boundary_coords, holes=holes, maxlength=1.5, is_optimized=True)

mesh = Mesh.from_vertices_and_faces(V, F)

# Visualize Initial Mesh
viewer = Viewer()
viewer.scene.add(mesh, name="Initial Mesh")
viewer.show()

## 3. Manipulate Mesh (Create Volcano Shape)

We identify vertices on the boundaries. Vertices on the inner holes are moved vertically and tilted to create the "volcano" openings.

In [ ]:
# We need to identify which vertices belong to the inner holes and which to the outer boundary.
# Using geometric distance is more robust than topology for this specific parametric setup.

fixed_vertices = set()

for v in mesh.vertices():
    if mesh.is_vertex_on_boundary(v):
        x, y, z = mesh.vertex_coordinates(v)
        # Distance from origin (2D)
        d_origin = (x**2 + y**2)**0.5
        
        if d_origin > 15.0:
            # Outer boundary
            fixed_vertices.add(v)
        else:
            # Inner boundary - find which hole it belongs to
            closest_hole = None
            min_dist = float('inf')
            
            for params in hole_params:
                hx, hy, hz = params['pos']
                dist = ((x - hx)**2 + (y - hy)**2)**0.5
                if dist < min_dist:
                    min_dist = dist
                    closest_hole = params
            
            if closest_hole:
                # Calculate tilt direction (outward from origin)
                hx, hy, hz = closest_hole['pos']
                
                # Radial vector from origin to hole center
                radial_vec = cg.Vector(hx, hy, 0)
                
                # Rotation axis is perpendicular to radial vector (tangent to circle)
                # Cross product with Z axis (0,0,1) gives tangent vector
                rot_axis = radial_vec.cross(cg.Vector(0, 0, 1))
                
                # Create rotation transformation
                # Angle in radians
                angle_rad = np.radians(closest_hole['tilt_angle'])
                R = cg.Rotation.from_axis_and_angle(rot_axis, angle_rad)
                
                # 1. Translate vertex to be relative to hole center
                v_local = cg.Point(x - hx, y - hy, 0)
                
                # 2. Apply rotation
                v_rotated = v_local.transformed(R)
                
                # 3. Translate back and apply height
                new_x = v_rotated.x + hx
                new_y = v_rotated.y + hy
                new_z = v_rotated.z + closest_hole['height']
                
                mesh.vertex_attributes(v, 'xyz', [new_x, new_y, new_z])
                fixed_vertices.add(v)

print(f"Fixed {len(fixed_vertices)} boundary vertices.")

# Visualize Manipulated Mesh
viewer = Viewer()
viewer.scene.add(mesh, name="Manipulated Mesh")
viewer.show()

## 4. Mesh Relaxation

We apply Laplacian smoothing to the mesh, keeping the boundary vertices fixed. This creates smooth transitions between the base and the raised/lowered holes.

In [ ]:
def relax_iteration(mesh, fixed, smoothing=0.5, load=0.0):
    """Apply one Laplacian smoothing step."""
    updates = {}
    for vertex in mesh.vertices():
        if vertex in fixed:
            continue

        nbrs = list(mesh.vertex_neighbors(vertex))
        if not nbrs:
            continue

        # Calculate centroid of neighbors
        cx, cy, cz = cg.centroid_points([mesh.vertex_coordinates(n) for n in nbrs])
        
        px, py, pz = mesh.vertex_coordinates(vertex)
        
        # Update position
        updates[vertex] = (
            px + smoothing * (cx - px),
            py + smoothing * (cy - py),
            pz + smoothing * (cz - pz) + load
        )

    for vertex, xyz in updates.items():
        mesh.vertex_attributes(vertex, "xyz", xyz)

# Run relaxation loop
for i in range(20):
    relax_iteration(mesh, fixed_vertices, smoothing=0.5, load=0.0)

# Visualize Relaxed Mesh
viewer = Viewer()
viewer.scene.add(mesh, name="Relaxed Mesh")
viewer.show()

## 5. Remeshing

Finally, we remesh the surface to ensure an even distribution of triangles, which improves the visual quality and suitability for further analysis or fabrication.

In [ ]:
# Convert to (V, F) for CGAL
V, F = mesh.to_vertices_and_faces()

# Remesh to get even triangulation
target_edge_length = 1
V, F = trimesh_remesh((V, F), target_edge_length, 20)

# Reconstruct mesh
mesh = Mesh.from_vertices_and_faces(V, F)

# Visualize Final Mesh
viewer = Viewer()
viewer.scene.add(mesh, name="Final Mesh", show_lines=True)
print(mesh.summary())
viewer.show()